<a href="https://colab.research.google.com/github/Ahnkyuwon504/AI-modeling/blob/main/instruction-tuning/gemma_ko_2b_summarize_instruction_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 개발 환경

## 1.1 라이브러리 설치

In [1]:
!pip3 install -q -U transformers==4.38.2
!pip3 install -q -U datasets==2.18.0
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.9.0
!pip3 install -q -U trl==0.7.11
!pip3 install -q -U accelerate==0.27.2

## 1.2 Import Modules

In [2]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

## 1.3 Huggingface 로그인

In [3]:
from huggingface_hub import notebook_login
notebook_login()

# 2. Dataset

## 2.1 데이터셋 로드

In [4]:
from datasets import load_dataset
dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")

## 2.2 데이터셋 탐색

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})

In [13]:
dataset['train'][11]

{'date': '2022-07-04 09:12:13',
 'category': 'economy',
 'press': '연합뉴스 ',
 'title': '특징주 삼성전자·SK하이닉스 또다시 52주 신저가 추락',
 'document': '삼성전자 SK하이닉스 CG 연합뉴스TV 제공 서울 연합뉴스 김아람 기자 삼성전자와 SK하이닉스가 4일 오전 재차 52주 신저가를 경신했다. 이날 오전 9시 5분 현재 유가증권시장에서 삼성전자는 전 거래일보다 0.53% 내린 5만5천900원에 거래 중이다. SK하이닉스도 전날보다 1.14% 내린 8만6천500원에 거래됐다. 개장 직후 삼성전자는 5만5천700원 SK하이닉스는 8만6천300원까지 낙폭을 키워 이틀 연속 52주 신저가를 새로 썼다. 최근 반도체 업황 우려가 확산하면서 두 종목 주가는 하락세를 이어왔다.',
 'link': 'https://n.news.naver.com/mnews/article/001/0013286226?sid=101',
 'summary': '4일 오전 9시 5분 현재 유가증권시장에서 삼성전자는 전 거래일보다 0.53% 내린 5만5천900원에, SK하이닉스도 전날보다 1.14% 내린 8만6천500원에 거래되면서 삼성전자와 SK하이닉스가 4일 오전 재차 52주 신저가를 경신했다.'}

In [6]:
dataset['train']['document'][11]

'삼성전자 SK하이닉스 CG 연합뉴스TV 제공 서울 연합뉴스 김아람 기자 삼성전자와 SK하이닉스가 4일 오전 재차 52주 신저가를 경신했다. 이날 오전 9시 5분 현재 유가증권시장에서 삼성전자는 전 거래일보다 0.53% 내린 5만5천900원에 거래 중이다. SK하이닉스도 전날보다 1.14% 내린 8만6천500원에 거래됐다. 개장 직후 삼성전자는 5만5천700원 SK하이닉스는 8만6천300원까지 낙폭을 키워 이틀 연속 52주 신저가를 새로 썼다. 최근 반도체 업황 우려가 확산하면서 두 종목 주가는 하락세를 이어왔다.'

# 3. Gemma모델 추론 테스트

## 3.1 모델 로드

In [7]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

BASE_MODEL = "google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

## 3.2 프롬프트

In [8]:
doc = dataset['train']['document'][11]

In [9]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128)

messages = [
    {
        "role": "user",
        "content": "다음 글을 요약해주세요.\n\n {}".format(doc)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [10]:
prompt

'<bos><start_of_turn>user\n다음 글을 요약해주세요.\n\n 삼성전자 SK하이닉스 CG 연합뉴스TV 제공 서울 연합뉴스 김아람 기자 삼성전자와 SK하이닉스가 4일 오전 재차 52주 신저가를 경신했다. 이날 오전 9시 5분 현재 유가증권시장에서 삼성전자는 전 거래일보다 0.53% 내린 5만5천900원에 거래 중이다. SK하이닉스도 전날보다 1.14% 내린 8만6천500원에 거래됐다. 개장 직후 삼성전자는 5만5천700원 SK하이닉스는 8만6천300원까지 낙폭을 키워 이틀 연속 52주 신저가를 새로 썼다. 최근 반도체 업황 우려가 확산하면서 두 종목 주가는 하락세를 이어왔다.<end_of_turn>\n<start_of_turn>model\n'

## 3-3 추론

In [11]:
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=150)

In [12]:
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
다음 글을 요약해주세요.

 삼성전자 SK하이닉스 CG 연합뉴스TV 제공 서울 연합뉴스 김아람 기자 삼성전자와 SK하이닉스가 4일 오전 재차 52주 신저가를 경신했다. 이날 오전 9시 5분 현재 유가증권시장에서 삼성전자는 전 거래일보다 0.53% 내린 5만5천900원에 거래 중이다. SK하이닉스도 전날보다 1.14% 내린 8만6천500원에 거래됐다. 개장 직후 삼성전자는 5만5천700원 SK하이닉스는 8만6천300원까지 낙폭을 키워 이틀 연속 52주 신저가를 새로 썼다. 최근 반도체 업황 우려가 확산하면서 두 종목 주가는 하락세를 이어왔다.<end_of_turn>
<start_of_turn>model
삼성전자와 SK하이닉스는 4일 오전 재차 52주 신저가를 경신했다. 삼성전자는 전 거래일보다 0.53% 내린 5만5천900원에 거래 중이고, SK하이닉스는 1.14% 내린 8만6천500원에 거래됐다. 개장 직후 삼성전자는 5만5천700원 SK하이닉스는 8만6천300원까지 낙폭을 키워 이틀 연속 52주 신저가를 새로 썼다.


In [14]:
print(dataset['train']['summary'][11])

4일 오전 9시 5분 현재 유가증권시장에서 삼성전자는 전 거래일보다 0.53% 내린 5만5천900원에, SK하이닉스도 전날보다 1.14% 내린 8만6천500원에 거래되면서 삼성전자와 SK하이닉스가 4일 오전 재차 52주 신저가를 경신했다.


# 4. instruction tuning

In [5]:
!nvidia-smi

Mon Jun 24 04:26:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## 4-1. 학습 데이터

In [8]:
prompt_curr = ""

def generate_prompt(example):
    prompt_list = []

    for i in range(len(example['document'])):
        prompt_curr = """
<bos>
<start_of_turn>user
다음 글을 요약해주세요:

{}
<end_of_turn>
<start_of_turn>model
{}
<end_of_turn>
<eos>
        """.format(example['document'][i], example['summary'][i])

        prompt_list.append(prompt_curr)

    return prompt_list

In [9]:
prompt_list = generate_prompt(dataset['train'][:1])

print(len(prompt_list))
print(prompt_list[0])

1

<bos>
<start_of_turn>user
다음 글을 요약해주세요:

앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.
<end_of_turn>
<start_of_turn>model
올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록한 가운데, 정부가 하반기에 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 결정

## 4-2. QLoRA(양자화) 적용 모델 로드

In [10]:
BASE_MODEL = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)
tokenizer.padding_side = 'right'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 4-3. 학습

In [11]:
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [12]:
train_data = dataset['train']

In [13]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
        max_steps=3000,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=0.03,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

Map:   0%|          | 0/22194 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [14]:
trainer.train()

Step,Training Loss
100,2.389000
200,2.062800
300,2.021200
400,1.912200
500,1.908000
600,1.903000
700,1.837400
800,1.831200
900,1.841400
1000,1.845300


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=3000, training_loss=1.770033889770508, metrics={'train_runtime': 8358.8121, 'train_samples_per_second': 1.436, 'train_steps_per_second': 0.359, 'total_flos': 6.766898603340595e+16, 'train_loss': 1.770033889770508, 'epoch': 0.54})

## 4-4. Finetuned Model 저장

In [15]:
ADAPTER_MODEL = "loar_adaptor"

trainer.model.save_pretrained(ADAPTER_MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
!ls -alh loar_adaptor

total 29M
drwxr-xr-x 2 root root 4.0K Jun 24 06:47 .
drwxr-xr-x 1 root root 4.0K Jun 24 06:47 ..
-rw-r--r-- 1 root root  690 Jun 24 06:47 adapter_config.json
-rw-r--r-- 1 root root  29M Jun 24 06:47 adapter_model.safetensors
-rw-r--r-- 1 root root 5.0K Jun 24 06:47 README.md


In [18]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained('gemma-2b-it-sum-ko')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
!ls -alh ./gemma-2b-it-sum-ko

total 4.7G
drwxr-xr-x 2 root root 4.0K Jun 24 06:48 .
drwxr-xr-x 1 root root 4.0K Jun 24 06:48 ..
-rw-r--r-- 1 root root  662 Jun 24 06:48 config.json
-rw-r--r-- 1 root root  132 Jun 24 06:48 generation_config.json
-rw-r--r-- 1 root root 4.7G Jun 24 06:48 model-00001-of-00002.safetensors
-rw-r--r-- 1 root root  65M Jun 24 06:48 model-00002-of-00002.safetensors
-rw-r--r-- 1 root root  14K Jun 24 06:48 model.safetensors.index.json


# 5. Finetuned 모델 추론

In [5]:
!nvidia-smi

Mon Jun 24 06:50:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## 5-1.모델 로드

In [6]:
BASE_MODEL = "google/gemma-2b-it"
FINETUNE_MODEL = "./gemma-2b-it-sum-ko"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## 5-2. 모델 추론

In [7]:
doc = dataset['test']['document'][10]

messages = [
    {
        "role": "user",
        "content": """
        다음 글을 요약해주세요:\n\n{}
        """.format(doc)
    }
]

pipe_finetuned = pipeline(
    "text-generation",
    model=finetune_model,
    tokenizer=tokenizer,
    max_new_tokens=512
)

prompt = pipe_finetuned.tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)

In [8]:
print(outputs[0]["generated_text"][len(prompt):])

한국동서발전은 다음달 31일까지 울산시민과 함께하는 생활 속 걷기 챌린지 탄소중립 건강걷기 누비GO 초록발자국 챌린지 를 진행한다고 1일 밝혔으며 울산숲사랑운동과 함께 하는 이번 챌린지는 코로나19로 지친 시민들이 일상 속 걷기 운동을 통해 건강을 증진하고 자가용 대신 대중교통을 이용해 탄소중립 사회를 실현하기 위해 마련됐다.




In [9]:
print(dataset['test']['summary'][10])

한국동서발전은 다음달 31일까지 울산숲사랑운동과 함께 코로나19로 지친 시민들이 일상 속 걷기 운동을 통해 건강을 증진하고 자가용 대신 대중교통을 이용해 탄소중립 사회를 실현하기 위한 챌린지 탄소중립 건강걷기 누비GO 초록발자국 챌린지 를 진행한다고 1일 밝혔다.
